# Rainy days on the stock market

Dataproject by Josefine Pedersen, Viktor Texel and Pernille Svendsen

> **Table of contents** 
> - Import and set magics
> - Introduction
> - Read and clean data from DMI and Yahoo Finance
> - Exploring the datasets
> - Merging the datasets
> - Analysis
> - Conclusion

*Imports and set magics:*

In [ ]:
# Import tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import requests # library for making HTTP requests
import datetime as dt # library for handling date and time objects
from PIL import Image
from scipy.stats import pearsonr
from skimage import io

# Autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# User written modules
import dataproject as dp

*Installation and import of data-specific tools and packages:*

**Note**: If the packages for importing data has not been installed, un-comment and run.

In [ ]:

# We install a package to inspect data from DMI (The Danish Meteorological Institute):
#%pip install dmi-open-data

# We install nescesarry packages from Yahoo Finance
#%pip install yfinance
#%pip install yahoofinancials

import yfinance as yf
from yahoofinancials import YahooFinancials

# Introduction

In this dataproject we wish to explore if there could be a correlation between weather and developments on the stock market. Through API's we import datasets from DMI and Yahoo Finance to examine whether or not there is a correlation between price fluctuations in the danish OMX C25-index and the amount of precipitation in Denmark. 

To do so, we carefully select comparable data from our datasets, clean it and analyze it using different python-tools. We find that there is litte-to-no correlation between precipitation and fluctuations in the share price of OMX C25 in Denmark.  

# Read and clean data from DMI and Yahoo Finance

In this section we import our data through API's and load it. To do so, the packages *dmi-open data*, *yfinance* and *yfinancials* needs to be installed. These are included at the top in our *import and set magics* section and can be installed by removing the '#' and then running the code. 

**We import data from DMI and print the available parameters**:

In [ ]:
# We use an API-key given to us from DMI's database to retrieve data
api_key = 'bd463c7d-f6f8-431d-a5a7-c466766a8363'
DMI_URL = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'

dp.import_data_dmi(api_key,DMI_URL)

From the parameters above, we select the parameter *precip_past1h* which returns the amount of precipitation the last hour in kg/m2 for a given period.

As we have now picked the necessary parameter for our analysis, we have to specify what timeperiod we wanna look at, which DMI-weather stations is relevant to include, and how to structure our DMI-dataset.

We wanna look at the most recent full year, 2022. We specify our timeperiod to start at the first hour of the first day of 2022, and end at the first hour of the first day of 2023. 

DMI has a lot of weatherstations scattered across Denmark, The Faroe Islands and Greenland. These weather stations have different capabilities, and some measure the same as others - meaning there is a risk of having measurements for a given geographical location twice. We only wanna look at *Pluvio* stations in Denmark, as these measure precipitation, cover the entirety of Denmark and are unique (only one station pr. location). Therefore we sort our data to only include *Pluvio* station IDs. 

The *Pluvio* stations are located as depicted below:

In [ ]:
image = io.imread('Pluvio.png')
io.imshow(image);

We select the data we need for our analysis and create the dataframe. Note that this takes a couple of minutes, since the API is set up such that we need to import the data from one station at a time.

In [ ]:
# Specifying the relevant parameter for the analysis
parameterId = ['precip_past1h']

# Creating the dataframe
df = dp.selecting_data_dmi(parameterId)
df.head()

To simplify the data we calculate the average of precipitation across all selected *pluvio* weather stations and store this data.

In [ ]:
# We create a row-average of the observations across weather stations
df['Precip'] = df.mean(axis=1, numeric_only = True)
df['Date'] = df.Time.dt.date 
df2 = df[['Time', 'Date', 'Precip']] 
df2

With 8760 hours in a year we have 22 missing hours. We investigate which days causes the issue.

In [ ]:
# We check to see which days have missing hours
check = df2.groupby(['Date'])['Time'].count()
check = pd.DataFrame(check)
check = check.loc[check['Time']!=24]
check

From the above table we can see, that for some days in June there is not data from all hours of the day. To tackle this issue, and to make our precipitation data more easily comparable, we calculate the daily average of the precipitation across weather stations (stored as df2) and store this data as df3. Now we have a dataset that displays the average daily precipitation across Denmark. 

In [ ]:
df3 = df2.groupby('Date').mean(numeric_only=True)
df3.reset_index(inplace=True)
df3['Date'] =pd.to_datetime(df3['Date'])
df3

**We import and clean data from Yahoo Finance**

In [25]:
# Setting the desired timeframe
from_time='2022-01-01'
to_time='2023-01-01'

# Importing dateframe
OMXC25 = dp.import_data_yahoo(from_time,to_time)
OMXC25

,Date,Close
0,2022-01-03,1970.949951
1,2022-01-04,1952.810059
2,2022-01-05,1947.630005
3,2022-01-06,1887.619995
4,2022-01-07,1867.989990
...,...,...
240,2022-12-23,1695.199951
241,2022-12-27,1701.630005
242,2022-12-28,1695.209961
243,2022-12-29,1712.839966


As can be seen from the table above, there are 250 observations/days where a closing price of OMX 25 have been noted by Yahoo Finance. This is due to the fact, that a closing price only occurs when the market is open. There are 260 weekdays in a year, and the 10 "missing" days are holidays.  

## Exploring each data set

Illustrating the average precipitation in 2022 and the closing price of OMXC25 in 2022:

In [ ]:
dp.explore_data(df3['Date'],
               df3['Precip'],
               'Average precipation in 2022',
               'Date','Precipation',0);

dp.explore_data(OMXC25['Date'],
               OMXC25['Close'],
               'Closing value of the price OMXC25',
               'Date','Closing price',1400);

Overall we can see that there are large fluctuations in both the average percipation and the closing price of OMXC25, however the fluctuations do not seem to be extremely correlated. To investigate further whether there is a correlation between the two, we merge the dataframes and analyze the data.

# Merge data sets

We combine our loaded data sets from DMI and Yahoo Finance.

In [ ]:
# Merge data
precip_stock = pd.merge(OMXC25, df3, on='Date', how='left')
precip_stock2 = precip_stock[['Date', 'Close', 'Precip']]
precip_stock2


We here use the left-join method to only keep dates where there have been registered a closing price for the stock. Therefore we want to drop all the values for average precipation on weekends and holidays. This is accomplished with this method for merging.

# Analysis

As we are mostly interested in the relative development in the stock price rather than the actual price, we calculate the pct. change from day-to-day to illustrate daily fluctuations. 

In [ ]:
# We calculate the pct. change from day-to-day on the closing price of OMXC25
precip_stock2['Change_in_stock'] = ((precip_stock2['Close'] / precip_stock2['Close'].shift(1) - 1)* 100)
precip_stock2.head()

We explore the merged data set to compare within the different months if we can se a correlation. This we do by firstly creating a variable for what month the observation belongs to, and then we plot interactively to study the months individually.

In [ ]:
precip_stock3 = precip_stock2.copy()
precip_stock3['Month'] = precip_stock3.Date.dt.month 

In [ ]:
# Making an interactive figure
def interactive_figure(Date, Month, Change_in_stock, Precip):
    
    filtered_data = precip_stock3[precip_stock3['Month']==Month]

    fig = plt.figure(figsize=(10,4))

    ax_left = fig.add_subplot(1,2,1)
    ax_left.set_title('Precipitation')
    ax_left.grid(True)
    ax_left.bar(filtered_data['Date'],filtered_data['Precip'])
    ax_left.set_xlabel('Date')
    ax_left.set_ylabel('mm')
    xticks = ax_left.get_xticks()
    ax_left.set_xticks(xticks[::len(xticks) // 4]) # set new tick positions
    ax_left.tick_params(axis='x', rotation=30) # set tick rotation

    ax_right = fig.add_subplot(1,2,2)
    ax_right.set_title('OMXC25')
    ax_right.grid(True)
    ax_right.bar(filtered_data['Date'],filtered_data['Change_in_stock'])
    ax_right.set_xlabel('Date')
    ax_right.set_ylabel('Share price')
    xticks2 = ax_right.get_xticks()
    ax_right.set_xticks(xticks2[::len(xticks2) // 4]) # set new tick positions
    ax_right.tick_params(axis='x', rotation=30) # set tick rotation

    plt.show()

widgets.interact(interactive_figure,
    Date=widgets.fixed(precip_stock3['Date']),
    Month=widgets.Dropdown(description="Month", options=precip_stock3.Month.unique(), value=1),
    Change_in_stock=widgets.fixed(precip_stock3['Change_in_stock']),
    Precip=widgets.fixed(precip_stock3['Precip'])
);

The interactive plots makes it possible to zoom in on one specific month and see how the share price of OMX C25 and the amount of precipitation differs from day to day in each month. 

However, when investigating the months we still don't see a strong correlation between precipation and changes in the stock prices.

As a way to more specifically see if there is a connection between the two variables, we wanna look at the days where the stock price fell/rose the most. Therefore, we utilize python to pick out the 10 days which saw the highest pct. change aswell as the 10 days with the biggest decrease. Our hypothesis is, that the amount of rainfall will be low, when the pct. change in stock is high and vice versa. 

We wanna compare whether the days with the largest increase in stock prices are coinciding with days where there was more rain than usual. Therefore we calculate the average precipation in 2022.

In [ ]:
precip_stock3['Average precip'] = precip_stock3['Precip'].mean(numeric_only=True)
precip_stock3['Precip - average'] = (precip_stock3['Precip'] - precip_stock3['Average precip'])

# 10 days with the highest increase in stock price
highest = precip_stock2.nlargest(10,'Change_in_stock') 

# 10 days with the largest decrease in stock price 
lowest = precip_stock2.nsmallest(10, 'Change_in_stock')

We investigate whether days with high increases in the stock price coincide with days with lower than average rain:

In [ ]:
highest

We see that 6 of the 10 observations have below average precipation, so it does somewhat support the hypothesis, but not to a large degree.

We do the same for the 10 days with largest decrease in the stock price:

In [ ]:
lowest

Here we see that only 3 out of 10 oberservations have above average precipation when there is a large fall in the stock price. So overall we do not see a large correlation between precipation and changes in the stock price.

Since there is not really any conclusive evidence in the above analysis, we now want to check if there is a correlation between the share price of the OMXC25 and the precipitation in 2022 for the entire dataframe. 

In [ ]:
# We drop the first row (not value for change in stock)
precip_stock4 = precip_stock2.iloc[1:]

dp.correlation(precip_stock4)

The coefficient shows a very weak positive correlation between the precipitation level and the changes in the OMXC25-index. Furthermore, the p-value shows that the correlation is not significant on a 5 pct. confidence level.

We can also see the weak correlation by plotting the observations:

In [ ]:
# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

ax.scatter(precip_stock4['Precip'],precip_stock4['Change_in_stock'])

ax.set_title('Scatterplot for precipation and change in stock price')
ax.set_xlabel('Precipation')
ax.set_ylabel('Change in stock price');

Overall the scatterplot above exhibits a very low correlation between the two variables.

# Conclusion

In this assignment we look at the precipitation level in Denmark in 2022 along with the OMXC25 index. We found that there was slim to no correlation between the variables meaning that the weather most likely does not affect the changes in share prices of the OMXC25. 

We come to this conclusion by using descriptive statistics and simple calculations. To investigate a possible correlation between stock prices and the weather satisfactionary, you would have to use much broader methods. The topic have already been analyzed by other economists, who take a look at the stock exchange in New York City. We refer to:

*Goetzmann, William N., and Ning Zhu. "Rain or shine: where is the weather effect?." European Financial Management 11.5 (2005): 559-578.*

*Saunders, Edward M. "Stock prices and Wall Street weather." The American Economic Review 83.5 (1993): 1337-1345.*

The first source states that there is low correlation between weather and stock prices, but the second source states that there to some degree is a correlation between the two. Overall, the conclusion regarding the subject is therefore somewhat ambigous. Our analysis, regarding Danish data, shows now correlation between rain and stock prices.